<a href="https://colab.research.google.com/github/saeedgolzar/AML_Course/blob/main/Project/Part_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applied Machine Learning Project

Saeed Golzar | Summer 2022

---

Import libraries

In [2]:
import os
import re
import random
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from zipfile import ZipFile
from google.colab import data_table
from difflib import SequenceMatcher
from sklearn.model_selection import train_test_split
data_table.enable_dataframe_formatter()

# Part 4: Pile Bearing Capacity by ML

In [3]:
!gdown 1BZLSTUM4f5G-yQA-G0lsboudJdDqUVGD -q

zf = ZipFile('/content/Records.zip', 'r')
zf.extractall()
zf.close()

In [4]:
CPT = dict()
directory = 'Records/CPTs'
for root, dirs, files in os.walk(directory):
  for filename in files:
    name = os.path.splitext(filename)[0][:-4]
    CPT[name] = pd.read_excel(os.path.join('/content/Records/CPTs/',filename), sheet_name=1, skiprows=1 , usecols=lambda x: 'Unnamed' not in x)

In [5]:
for key in CPT:
  try: 
    CPT[key] = CPT[key][['Z (m)', 'qc (MPa)', 'fs (MPa)']]
  except:
    CPT[key] = CPT[key][['Z (m)', 'qt (MPa)', 'fs (MPa)']]

In [6]:
!gdown 1gpexAg2ia6NU8zO2d7-B0N45bmiV0Uh2 -q

df = pd.read_excel('/content/CPT-Data.xlsx')

In [7]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [8]:
df['FileName'] = df.apply(lambda _: '', axis=1)

if __name__ == '__main__':
  for index, row in df.iterrows():
    sim = 0
    for key in CPT:
      if similar(df['Case ID'][index], key) > sim:
        sim = similar(df['Case ID'][index], key)
        filename = key
      if df['Case ID'][index] in key:
        filename = key
    df['FileName'][index] = filename

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [9]:
df = df[['FileName'] + [col for col in df.columns if col != 'FileName']]
df = df[['Case ID'] + [col for col in df.columns if col != 'Case ID']]

In [10]:
df.head(10)

,Case ID,FileName,Case No,PY,Pile No,Shape,Material,"Embedment Length, D (m)",Installation,"Cross Sectional Area, At (m2)",Equivalent Diameters,"Total Capacity, Ru, kN (Test No1)",GWL (m),CPT Depth (m),qc,fs,u,Sand,Clay,Mix
0,001-A&M1,001-A&M14-H,2.0,1996,1,Square,Concrete,8.80,Driven,0.160000,400.0,1140.0,0.0,14.00,True,True,False,False,False,True
1,001-A&M40,001-A&M40,21.0,1996,21,Square,Concrete,16.00,Driven,0.122500,350.0,1070.0,0.0,24.50,True,True,False,True,False,False
2,001-A&M87,001-A&M87,36.0,1996,36,Square,Concrete,7.40,Driven,0.160000,400.0,640.0,0.0,13.33,True,True,False,False,True,False
3,001-A&N2,001-A&M1,42.0,1996,42,Square,Concrete,13.75,Driven,0.202500,450.0,4250.0,0.0,17.00,True,True,False,True,False,False
4,001-BGHD1,001-BGHD1,45.0,1996,45,Square,Concrete,11.00,Driven,0.081225,285.0,1000.0,6.0,20.00,True,True,False,True,False,False
5,001-KP1,001-KP1-H,63.0,1996,63,H Pile,Steel,14.00,Driven,0.046000,400.0,3500.0,1.0,17.00,True,True,False,True,False,False
6,001-L&D37,001-L&D37,75.0,1996,75,Pipe,Steel,14.60,Driven,0.125664,400.0,1800.0,0.0,19.00,True,True,False,True,False,False
7,001-LSUB12,001-LSUB12,81.0,1996,81,Round,Concrete,37.80,Driven,0.636173,900.0,3960.0,0.0,42.40,True,True,False,False,False,True
8,001-LTN484,001-LTN484,90.0,1996,90,Round,Concrete,7.60,Bored,0.159043,450.0,750.0,0.0,11.40,True,True,False,False,False,True
9,001-N&SBI43,001-N&SBI43,102.0,1996,102,Pipe,Steel,22.50,Driven,0.058535,273.0,1620.0,0.0,24.60,True,True,False,True,False,False


In [35]:
df = df.dropna(subset=['Embedment Length, D (m)'])
df = df.dropna(subset=['Equivalent Diameters'])

In [39]:
def cpt_feature_extractor(cpt, L, D):
  fss = list()
  qcc = list()
  for index in cpt.index:
    if cpt['Z (m)'][index] < L:
      fss.append(cpt['fs (MPa)'][row])
    if cpt['Z (m)'][index] < (L + D) & cpt['Z (m)'][index] > (L - D):
      qcc.append(cpt['fs (MPa)'][index])
  cpt_t = sum(qcc) / len(qcc)
  cpt_s = sum(fss) / len(fss)
  return cpt_t, cpt_s

In [ ]:
df['CPT_s'] = df.apply(lambda _: '', axis=1)
df['CPT_t'] = df.apply(lambda _: '', axis=1)


for index in df.index:
 
  print(CPT[df['FileName'][index]], df['Embedment Length, D (m)'][index], df['Equivalent Diameters'][index])

  df['CPT_s'][index] = cpt_s
  
  df['CPT_t'][index] = cpt_t

In [ ]:
X = df [['CPT_s', 'CPT_t', 'Embedment Length, D (m)', 'Equivalent Diameters', 'Material']]

y = df ['Total Capacity, Ru, kN (Test No1)']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=20, input_dim=111, activation="relu"))
model.add(tf.keras.layers.Dense(units=100, activation="relu"))
model.add(tf.keras.layers.Dense(units=100, activation="relu"))
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
model.compile(optimizer='adam',
            loss="binary_crossentropy",
            metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), validation_freq=1)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)